In [12]:
polish_dictionary = {}
with open("morfologik.txt") as f:
    for line in f.readlines():
        base_word, form, meta = line.split(";")
        polish_dictionary[form] = base_word
        
print(len(polish_dictionary))

4668625


In [15]:
import regex
word_pattern = "\p{Letter}+"

In [50]:
from collections import defaultdict
words = defaultdict(lambda: 0)
not_found = defaultdict(lambda: 0)

In [51]:
import os, json

import pickle
from tqdm import tqdm

FILE_LIST = "files.pickle"
DATA_DIR = "/run/media/maciej/Nowy/data/json/"
CHOSEN_YEAR = "2011"

def update_dicts(word):
    if word in polish_dictionary:
        base_form = polish_dictionary[word]
        words[base_form] += 1
    elif word.lower() in polish_dictionary:
        base_form = polish_dictionary[word.lower()]
        words[base_form] += 1
    else:
        not_found[word] += 1

def load_data():    
    files = pickle.load(open(FILE_LIST, 'rb'))
    for file in tqdm(files):
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [x["textContent"] for x in data["items"] if x["judgmentDate"].startswith(CHOSEN_YEAR)]

            for judgment in judgments:
                judgment = regex.sub("\<.*?\>", "", judgment, regex.DOTALL)
                
                for match in regex.finditer(word_pattern, judgment):
                    [word] = match.captures()
                    update_dicts(word)
            
load_data()

100%|██████████| 68/68 [00:18<00:00,  3.71it/s]


In [52]:
sorted(not_found.items(), key=lambda x: x[1], reverse=True)

[('span', 110015),
 ('class', 54985),
 ('anon', 54625),
 ('block', 54625),
 ('c', 47255),
 ('Pzp', 31572),
 ('td', 24174),
 ('gov', 20055),
 ('href', 19959),
 ('isap', 19959),
 ('DetailsServlet', 19959),
 ('WDU', 19959),
 ('title', 19959),
 ('SIWZ', 19106),
 ('sygn', 15681),
 ('zw', 11067),
 ('Sygn', 10534),
 ('strong', 10284),
 ('tr', 8342),
 ('J', 7653),
 ('siwz', 7264),
 ('sup', 6317),
 ('SSN', 6102),
 ('C', 5923),
 ('OTK', 4901),
 ('L', 4474),
 ('IV', 4439),
 ('ZU', 4353),
 ('SA', 3804),
 ('CSK', 3561),
 ('h', 3316),
 ('LEX', 3198),
 ('pzp', 3145),
 ('j', 3133),
 ('UZP', 2989),
 ('OSNC', 2752),
 ('SK', 2729),
 ('H', 2527),
 ('CZP', 2218),
 ('l', 2176),
 ('div', 2064),
 ('ppkt', 2032),
 ('niepubl', 1987),
 ('CKN', 1923),
 ('OSNP', 1906),
 ('dd', 1859),
 ('dt', 1850),
 ('VI', 1696),
 ('późn', 1647),
 ('UK', 1646),
 ('F', 1538),
 ('PLN', 1519),
 ('UOKiK', 1509),
 ('publ', 1473),
 ('x', 1460),
 ('Poland', 1451),
 ('zd', 1424),
 ('Ł', 1397),
 ('PZP', 1116),
 ('Asseco', 1109),
 ('Lex', 9